## INSTALLING THE DEPENDENCIES

In [1]:
!pip install sqlalchemy #install the sqlalchemy package
!pip install psycopg2 #install the postgres driver package

You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## CONNECTING TO THE POSTGRES geekgap_demos DATABASE

In [2]:
DB_URI = 'postgresql+psycopg2://postgres:@localhost/geekgap_webinar1'

from sqlalchemy import create_engine
engine = create_engine(DB_URI, echo=True)

/Users/junior/.virtualenvs/ustem/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## CREATING AN ORM MAPPING
- ORM = Object Relational Mapping
- Looks like coding an ERD Diagram instead of drawing one

In [3]:
import datetime

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, DateTime, VARCHAR, ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

In [4]:
class Customer(Base):
    __tablename__ = 'Customer'
    
    customer_ID = Column(Integer, primary_key=True)
    first_name = Column(VARCHAR(50))
    last_name = Column(VARCHAR(50))
    address = Column(VARCHAR(200))
    email = Column(VARCHAR(50))
    phone = Column(VARCHAR(10))
    
    # One(Customer)-To-Many(Orders) relationship
    orders = relationship('Order')
    
    def __repr__(self):
        return f"<Customer(first_name={self.first_name}, last_name={self.last_name})>"  

In [5]:
Customer.__table__

Table('Customer', MetaData(bind=None), Column('customer_ID', Integer(), table=<Customer>, primary_key=True, nullable=False), Column('first_name', VARCHAR(length=50), table=<Customer>), Column('last_name', VARCHAR(length=50), table=<Customer>), Column('address', VARCHAR(length=200), table=<Customer>), Column('email', VARCHAR(length=50), table=<Customer>), Column('phone', VARCHAR(length=10), table=<Customer>), schema=None)

In [6]:
# Many(Products)-To-Many(Orders) assiociation table
class Association__Product_Order(Base):
    __tablename__ = 'Association__Product_Order'
    product_ID = Column('product_ID', Integer, ForeignKey('Product.product_ID'), primary_key=True)
    order_ID = Column('order_ID', Integer, ForeignKey('Order.order_ID'), primary_key=True)
    
    product_quantity = Column(Integer) # attribute carried by the association

    product = relationship("Product", back_populates="orders")
    order = relationship("Order", back_populates="products")


class Order(Base):
    __tablename__ = 'Order'
    
    order_ID = Column(Integer, primary_key=True)
    order_date = Column(DateTime,  default=datetime.datetime.utcnow)
    
    # One(Customer)-To-Many(Orders) relationship
    customer_ID = Column(Integer,ForeignKey('Customer.customer_ID')) 
    customer = relationship("Customer", back_populates="orders")
    
    # Many(Products)-To-Many(Orders) relationship
    products = relationship("Association__Product_Order", back_populates="order")
    
    # one(Delivery) to One(Order) relationship
    delivery = relationship("Delivery", uselist=False, back_populates='order')
    
    
    def __repr__(self):
        return f"<Order(order_ID={self.order_ID}, order_date={self.order_date})>"  
    
    
class Product(Base):
    __tablename__ = 'Product'
    
    product_ID = Column(Integer, primary_key=True)
    name = Column(VARCHAR(50), index=True)
    product_type = Column(VARCHAR(50))
    quantity = Column(Integer)
    
    # Many(Products)-To-Many(Orders) relationship
    orders = relationship('Association__Product_Order', back_populates="product")
    
    def __repr__(self):
        return f"<Product(name={self.name}, type={self.product_type})>"  

In [7]:
Order.__table__

Table('Order', MetaData(bind=None), Column('order_ID', Integer(), table=<Order>, primary_key=True, nullable=False), Column('order_date', DateTime(), table=<Order>, default=ColumnDefault(<function datetime.utcnow at 0x10f90a378>)), Column('customer_ID', Integer(), ForeignKey('Customer.customer_ID'), table=<Order>), schema=None)

In [8]:
Product.__table__

Table('Product', MetaData(bind=None), Column('product_ID', Integer(), table=<Product>, primary_key=True, nullable=False), Column('name', VARCHAR(length=50), table=<Product>), Column('product_type', VARCHAR(length=50), table=<Product>), Column('quantity', Integer(), table=<Product>), schema=None)

In [9]:
class Delivery(Base):
    __tablename__ = 'Delivery'
    
    delivery_ID = Column(Integer, primary_key=True)
    delivery_date = Column(DateTime,  default=datetime.datetime.utcnow)
    delivery_address = Column(VARCHAR(200))
    
    # one(Delivery) to One(Order) relationship
    order_id = Column(Integer, ForeignKey('Order.order_ID'))
    order = relationship('Order', back_populates="delivery")
    
    def __repr__(self):
        return f"<Delivery(oder_id={self.order_id}, date={self.delivery_date}, address={self.delivery_address})>"  

In [11]:
Delivery.__table__

Table('Delivery', MetaData(bind=None), Column('delivery_ID', Integer(), table=<Delivery>, primary_key=True, nullable=False), Column('delivery_date', DateTime(), table=<Delivery>, default=ColumnDefault(<function datetime.utcnow at 0x10d1c38c8>)), Column('delivery_address', VARCHAR(length=200), table=<Delivery>), Column('order_id', Integer(), ForeignKey('Order.order_ID'), table=<Delivery>), schema=None)

## CREATE THE DATABASE SCHEMA

In [12]:
Base.metadata.create_all(engine)

2019-10-26 18:50:16,352 INFO sqlalchemy.engine.base.Engine select version()
2019-10-26 18:50:16,353 INFO sqlalchemy.engine.base.Engine {}
2019-10-26 18:50:16,357 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-10-26 18:50:16,357 INFO sqlalchemy.engine.base.Engine {}
2019-10-26 18:50:16,360 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-10-26 18:50:16,361 INFO sqlalchemy.engine.base.Engine {}
2019-10-26 18:50:16,376 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-10-26 18:50:16,376 INFO sqlalchemy.engine.base.Engine {}
2019-10-26 18:50:16,378 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-10-26 18:50:16,380 INFO sqlalchemy.engine.base.Engine {}
2019-10-26 18:50:16,382 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

## CREATING A SESSION

In [13]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [14]:
session = Session()

## ADDING PRODUCTS

In [15]:
p1 = Product(name='Computer', product_type='tech', quantity=50)
p2 = Product(name='Chair', product_type='furniture', quantity=100)
p3 = Product(name='pencil', product_type='eduction', quantity=1000)
p4 = Product(name='bread', product_type='food', quantity=5)
p5 = Product(name='fish', product_type='food', quantity=2)
p6 = Product(name='movie', product_type='entertainment', quantity=500)

session.add_all([p1, p2, p3, p4, p5, p6])

In [16]:
p1

<Product(name=Computer, type=tech)>

In [19]:
p1.product_ID # returns None as the product has not yet been persisted in the Database with commit

In [20]:
session.commit()

2019-10-26 18:53:10,646 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 18:53:10,649 INFO sqlalchemy.engine.base.Engine INSERT INTO "Product" (name, product_type, quantity) VALUES (%(name)s, %(product_type)s, %(quantity)s) RETURNING "Product"."product_ID"
2019-10-26 18:53:10,651 INFO sqlalchemy.engine.base.Engine {'name': 'Computer', 'product_type': 'tech', 'quantity': 50}
2019-10-26 18:53:10,678 INFO sqlalchemy.engine.base.Engine INSERT INTO "Product" (name, product_type, quantity) VALUES (%(name)s, %(product_type)s, %(quantity)s) RETURNING "Product"."product_ID"
2019-10-26 18:53:10,680 INFO sqlalchemy.engine.base.Engine {'name': 'Chair', 'product_type': 'furniture', 'quantity': 100}
2019-10-26 18:53:10,682 INFO sqlalchemy.engine.base.Engine INSERT INTO "Product" (name, product_type, quantity) VALUES (%(name)s, %(product_type)s, %(quantity)s) RETURNING "Product"."product_ID"
2019-10-26 18:53:10,682 INFO sqlalchemy.engine.base.Engine {'name': 'pencil', 'product_type': 'e

In [21]:
p1.product_ID # after commit running the same query should return the product_id

2019-10-26 18:53:22,163 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 18:53:22,165 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type", "Product".quantity AS "Product_quantity" 
FROM "Product" 
WHERE "Product"."product_ID" = %(param_1)s
2019-10-26 18:53:22,166 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


1

In [22]:
p4.quantity = 10

In [23]:
session.commit()

2019-10-26 18:54:02,085 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type" 
FROM "Product" 
WHERE "Product"."product_ID" = %(param_1)s
2019-10-26 18:54:02,086 INFO sqlalchemy.engine.base.Engine {'param_1': 4}
2019-10-26 18:54:02,089 INFO sqlalchemy.engine.base.Engine UPDATE "Product" SET quantity=%(quantity)s WHERE "Product"."product_ID" = %(Product_product_ID)s
2019-10-26 18:54:02,090 INFO sqlalchemy.engine.base.Engine {'quantity': 10, 'Product_product_ID': 4}
2019-10-26 18:54:02,092 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
p4.quantity

2019-10-26 18:54:19,483 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 18:54:19,485 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type", "Product".quantity AS "Product_quantity" 
FROM "Product" 
WHERE "Product"."product_ID" = %(param_1)s
2019-10-26 18:54:19,487 INFO sqlalchemy.engine.base.Engine {'param_1': 4}


10

## ADDING CUSTOMERS

In [25]:
c1 = Customer(first_name='Junior', last_name='Teudjio',
              email='junior.teudjio@test.com', address='London', phone='12345')
session.add(c1)

c2 = Customer(first_name='John', last_name='Doe',
              email='john.doe@test.com', address='Paris', phone='12344')
session.add(c2)

In [26]:
c1

<Customer(first_name=Junior, last_name=Teudjio)>

In [27]:
session.commit()

2019-10-26 18:54:43,312 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (first_name, last_name, address, email, phone) VALUES (%(first_name)s, %(last_name)s, %(address)s, %(email)s, %(phone)s) RETURNING "Customer"."customer_ID"
2019-10-26 18:54:43,313 INFO sqlalchemy.engine.base.Engine {'first_name': 'Junior', 'last_name': 'Teudjio', 'address': 'London', 'email': 'junior.teudjio@test.com', 'phone': '12345'}
2019-10-26 18:54:43,347 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (first_name, last_name, address, email, phone) VALUES (%(first_name)s, %(last_name)s, %(address)s, %(email)s, %(phone)s) RETURNING "Customer"."customer_ID"
2019-10-26 18:54:43,348 INFO sqlalchemy.engine.base.Engine {'first_name': 'John', 'last_name': 'Doe', 'address': 'Paris', 'email': 'john.doe@test.com', 'phone': '12344'}
2019-10-26 18:54:43,350 INFO sqlalchemy.engine.base.Engine COMMIT


## ADDING ORDERS AND DELIVERIES

In [28]:
# customer1 initiating the order1
o1 = Order()
o1.customer = c1

In [29]:
# customer 1 adding product1 to his order
a_po1 = Association__Product_Order(product_quantity=5)
a_po1.order = o1
a_po1.product = p1

In [30]:
session.add(o1)

In [31]:
session.commit()

2019-10-26 18:56:19,148 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 18:56:19,151 INFO sqlalchemy.engine.base.Engine SELECT "Customer"."customer_ID" AS "Customer_customer_ID", "Customer".first_name AS "Customer_first_name", "Customer".last_name AS "Customer_last_name", "Customer".address AS "Customer_address", "Customer".email AS "Customer_email", "Customer".phone AS "Customer_phone" 
FROM "Customer" 
WHERE "Customer"."customer_ID" = %(param_1)s
2019-10-26 18:56:19,152 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2019-10-26 18:56:19,156 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type", "Product".quantity AS "Product_quantity" 
FROM "Product" 
WHERE "Product"."product_ID" = %(param_1)s
2019-10-26 18:56:19,157 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2019-10-26 18:56:19,162 INFO sqlalchemy.engine.base.Engine INSERT INTO "Order" (order

In [43]:
# customer1 initiating the order1
o3 = Order()
o3.customer = c1

In [44]:
# customer 1 adding product1 to his order
a_po3 = Association__Product_Order(product_quantity=100)
a_po3.order = o1
a_po3.product = p1

In [45]:
session.add(o3)
session.add(a_po3)

In [46]:
session.commit()

2019-10-26 19:02:56,487 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 19:02:56,490 INFO sqlalchemy.engine.base.Engine SELECT "Customer"."customer_ID" AS "Customer_customer_ID", "Customer".first_name AS "Customer_first_name", "Customer".last_name AS "Customer_last_name", "Customer".address AS "Customer_address", "Customer".email AS "Customer_email", "Customer".phone AS "Customer_phone" 
FROM "Customer" 
WHERE "Customer"."customer_ID" = %(param_1)s
2019-10-26 19:02:56,491 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2019-10-26 19:02:56,494 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type", "Product".quantity AS "Product_quantity" 
FROM "Product" 
WHERE "Product"."product_ID" = %(param_1)s
2019-10-26 19:02:56,495 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2019-10-26 19:02:56,498 INFO sqlalchemy.engine.base.Engine INSERT INTO "Order" (order

FlushError: New instance <Association__Product_Order at 0x10f9f6128> with identity key (<class '__main__.Association__Product_Order'>, (1, 1), None) conflicts with persistent instance <Association__Product_Order at 0x10f9b8780>

## QUERYING

In [32]:
session.query(Product).filter(Product.name.in_(['fish', 'bread'])).all()

2019-10-26 18:57:23,484 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 18:57:23,486 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type", "Product".quantity AS "Product_quantity" 
FROM "Product" 
WHERE "Product".name IN (%(name_1)s, %(name_2)s)
2019-10-26 18:57:23,488 INFO sqlalchemy.engine.base.Engine {'name_1': 'fish', 'name_2': 'bread'}


[<Product(name=fish, type=food)>, <Product(name=bread, type=food)>]

In [34]:
session.query(Product).filter(Product.product_type=='food').all()

2019-10-26 18:58:01,705 INFO sqlalchemy.engine.base.Engine SELECT "Product"."product_ID" AS "Product_product_ID", "Product".name AS "Product_name", "Product".product_type AS "Product_product_type", "Product".quantity AS "Product_quantity" 
FROM "Product" 
WHERE "Product".product_type = %(product_type_1)s
2019-10-26 18:58:01,706 INFO sqlalchemy.engine.base.Engine {'product_type_1': 'food'}


[<Product(name=fish, type=food)>, <Product(name=bread, type=food)>]

In [35]:
session.query(Order).filter(Order.customer==c1).order_by(Order.order_date).all()

2019-10-26 18:58:28,517 INFO sqlalchemy.engine.base.Engine SELECT "Customer"."customer_ID" AS "Customer_customer_ID", "Customer".first_name AS "Customer_first_name", "Customer".last_name AS "Customer_last_name", "Customer".address AS "Customer_address", "Customer".email AS "Customer_email", "Customer".phone AS "Customer_phone" 
FROM "Customer" 
WHERE "Customer"."customer_ID" = %(param_1)s
2019-10-26 18:58:28,519 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2019-10-26 18:58:28,522 INFO sqlalchemy.engine.base.Engine SELECT "Order"."order_ID" AS "Order_order_ID", "Order".order_date AS "Order_order_date", "Order"."customer_ID" AS "Order_customer_ID" 
FROM "Order" 
WHERE %(param_1)s = "Order"."customer_ID" ORDER BY "Order".order_date
2019-10-26 18:58:28,524 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[<Order(order_ID=1, order_date=2019-10-26 17:56:19.162482)>]

In [36]:
session.query(Order).filter(Order.customer==c2).order_by(Order.order_date).all()

2019-10-26 18:58:46,284 INFO sqlalchemy.engine.base.Engine SELECT "Customer"."customer_ID" AS "Customer_customer_ID", "Customer".first_name AS "Customer_first_name", "Customer".last_name AS "Customer_last_name", "Customer".address AS "Customer_address", "Customer".email AS "Customer_email", "Customer".phone AS "Customer_phone" 
FROM "Customer" 
WHERE "Customer"."customer_ID" = %(param_1)s
2019-10-26 18:58:46,286 INFO sqlalchemy.engine.base.Engine {'param_1': 2}
2019-10-26 18:58:46,289 INFO sqlalchemy.engine.base.Engine SELECT "Order"."order_ID" AS "Order_order_ID", "Order".order_date AS "Order_order_date", "Order"."customer_ID" AS "Order_customer_ID" 
FROM "Order" 
WHERE %(param_1)s = "Order"."customer_ID" ORDER BY "Order".order_date
2019-10-26 18:58:46,291 INFO sqlalchemy.engine.base.Engine {'param_1': 2}


[]